In [1]:
from Neuroradiomics.registration import registration_reader
from Neuroradiomics.registration import elastix_multimap_registration
from Neuroradiomics.registration import registration_writer
from Neuroradiomics.registration import apply_transform_from_files
from Neuroradiomics.registration import read_transform_from_files
from Neuroradiomics.registration import elastix_rigid_registration
from Neuroradiomics.registration import Set_sampler_parameters_as_image
from Neuroradiomics.registration import elastix_transform_parameters_writer
from Neuroradiomics.registration import transform_parameters_writer
from Neuroradiomics.registration import evaluate_registration_mse



import itk
import numpy as np
import os

import tkinter as tk
from tkinter import filedialog

In [2]:
root = tk.Tk()
root.withdraw()


''

In [4]:
fixed_image_filename = filedialog.askopenfilename(title = "Select the fixed image")

    #Moving Image
moving_image_filename = filedialog.askopenfilename(title = "Select the moving image")

    #Where to save the registered image
#output_filepath = filedialog.askdirectory(title = "Select where do you want to save the new image")

In [5]:
fix_im, mov_im = registration_reader(fixed_image_filename, moving_image_filename)

In [ ]:
reg_obj = elastix_rigid_registration(fix_im, mov_im, True)

In [6]:
atlas_filename = filedialog.askopenfilename(title = "Select the moving image")

In [7]:
atlas = itk.imread(atlas_filename)

In [8]:
params = Set_sampler_parameters_as_image(reg_obj.GetTransformParameterObject(), mov_im)

In [9]:
right_t1 = itk.transformix_filter(mov_im, params)

In [ ]:
reg_atlas = elastix_multimap_registration(right_t1, atlas, True)

In [12]:
evaluate_registration_mse(right_t1, reg_atlas.GetOutput())

0.031343687

In [15]:
evaluate_registration_mse(fix_im, reg_obj.GetOutput())

0.006375213